In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, merge, Activation 
from keras.layers import Conv2D, Concatenate, BatchNormalization
from keras.layers import MaxPooling2D, AveragePooling2D, SeparableConv2D
from keras.optimizers import Adam
from google.colab import files

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 12  # #layers_per_block(12) = (depth(40) - (total_blocks(3) + 1)) / total_blocks(3). It will further divided by 2 in case of DenseNet-BC"
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]


170500096/170498071 [==============================] - 12s 0us/step


In [0]:

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block with RMSA kernel Initializer
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):

# Bottlenack layer with compression in dense block. Accuracy : 81.21

#         BatchNorm = BatchNormalization()(temp)
#         relu = Activation('relu')(BatchNorm)
#         Conv2D_1_1 = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='valid', kernel_initializer='VarianceScaling')(relu)
#         if dropout_rate>0:
#           Conv2D_1_1 = Dropout(dropout_rate)(Conv2D_1_1)
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)

# Tried Depthwise Separable Convolution in place of Conv2D. Accuracy : 71.43

        # Removed Compression from Conv2D.
        Conv2D_3_3 = Conv2D(int(num_filter), (3,3), use_bias=False ,padding='same', kernel_initializer='VarianceScaling')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same', kernel_initializer='VarianceScaling')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2),strides=2)(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same', kernel_initializer='VarianceScaling')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 12)   324         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 12)   48          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 12)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
model.save_weights("DNST_model_1.h5")
files.download('DNST_model_1.h5')


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 245s 5ms/step - loss: 1.5934 - acc: 0.4009 - val_loss: 1.6965 - val_acc: 0.4250
Epoch 2/10
19200/50000 [==========>...................] - ETA: 2:13 - loss: 1.3137 - acc: 0.5106

50000/50000 [==============================] - 231s 5ms/step - loss: 1.2478 - acc: 0.5404 - val_loss: 1.6655 - val_acc: 0.5044
Epoch 3/10
45568/50000 [==========================>...] - ETA: 19s - loss: 1.0927 - acc: 0.6029

50000/50000 [==============================] - 231s 5ms/step - loss: 1.0868 - acc: 0.6055 - val_loss: 1.3143 - val_acc: 0.5754
Epoch 4/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.9896 - acc: 0.6444 - val_loss: 1.4452 - val_acc: 0.5558
Epoch 5/10
 3328/50000 [>.............................] - ETA: 3:22 - loss: 0.9547 - acc: 0.6566

50000/50000 [==============================] - 231s 5ms/step - loss: 0.9242 - acc: 0.6692 - val_loss: 2.3199 - val_acc: 0.4165
Epoch 6/10
39424/50000 [======================>.......] - ETA: 46s - loss: 0.8659 - acc: 0.6918

50000/50000 [==============================] - 231s 5ms/step - loss: 0.8632 - acc: 0.6926 - val_loss: 0.9315 - val_acc: 0.6847
Epoch 7/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.8186 - acc: 0.7067 - val_loss: 1.2566 - val_acc: 0.6388
Epoch 8/10
 1792/50000 [>.............................] - ETA: 3:30 - loss: 0.8022 - acc: 0.7137

50000/50000 [==============================] - 231s 5ms/step - loss: 0.7734 - acc: 0.7254 - val_loss: 1.0976 - val_acc: 0.6691
Epoch 9/10
38912/50000 [======================>.......] - ETA: 48s - loss: 0.7448 - acc: 0.7350

50000/50000 [==============================] - 231s 5ms/step - loss: 0.7432 - acc: 0.7352 - val_loss: 1.0269 - val_acc: 0.6869
Epoch 10/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.7103 - acc: 0.7480 - val_loss: 0.8772 - val_acc: 0.7243


In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
model.save_weights("DNST_model_2.h5")
files.download('DNST_model_2.h5')


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.6755 - acc: 0.7614 - val_loss: 0.9277 - val_acc: 0.7000
Epoch 2/10
19200/50000 [==========>...................] - ETA: 2:13 - loss: 0.6578 - acc: 0.7682

50000/50000 [==============================] - 231s 5ms/step - loss: 0.6554 - acc: 0.7675 - val_loss: 1.0595 - val_acc: 0.6829
Epoch 3/10
45568/50000 [==========================>...] - ETA: 19s - loss: 0.6307 - acc: 0.7763

50000/50000 [==============================] - 230s 5ms/step - loss: 0.6315 - acc: 0.7759 - val_loss: 0.8630 - val_acc: 0.7283
Epoch 4/10
50000/50000 [==============================] - 229s 5ms/step - loss: 0.6100 - acc: 0.7857 - val_loss: 0.7965 - val_acc: 0.7489
Epoch 5/10
 3328/50000 [>.............................] - ETA: 3:21 - loss: 0.5797 - acc: 0.7951

50000/50000 [==============================] - 230s 5ms/step - loss: 0.5944 - acc: 0.7924 - val_loss: 0.8274 - val_acc: 0.7391
Epoch 6/10
39424/50000 [======================>.......] - ETA: 45s - loss: 0.5734 - acc: 0.7984

50000/50000 [==============================] - 230s 5ms/step - loss: 0.5769 - acc: 0.7977 - val_loss: 0.9027 - val_acc: 0.7307
Epoch 7/10
50000/50000 [==============================] - 229s 5ms/step - loss: 0.5531 - acc: 0.8062 - val_loss: 0.7071 - val_acc: 0.7773
Epoch 8/10
 1792/50000 [>.............................] - ETA: 3:27 - loss: 0.5334 - acc: 0.8142

50000/50000 [==============================] - 230s 5ms/step - loss: 0.5457 - acc: 0.8065 - val_loss: 0.8843 - val_acc: 0.7308
Epoch 9/10
38912/50000 [======================>.......] - ETA: 47s - loss: 0.5275 - acc: 0.8141

50000/50000 [==============================] - 229s 5ms/step - loss: 0.5316 - acc: 0.8133 - val_loss: 0.6614 - val_acc: 0.7862
Epoch 10/10
50000/50000 [==============================] - 230s 5ms/step - loss: 0.5154 - acc: 0.8190 - val_loss: 0.8169 - val_acc: 0.7582


In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
model.save_weights("DNST_model_3.h5")
files.download('DNST_model_3.h5')


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 230s 5ms/step - loss: 0.5065 - acc: 0.8215 - val_loss: 0.8531 - val_acc: 0.7521
Epoch 2/10
19200/50000 [==========>...................] - ETA: 2:13 - loss: 0.4802 - acc: 0.8329

50000/50000 [==============================] - 230s 5ms/step - loss: 0.4954 - acc: 0.8273 - val_loss: 1.2509 - val_acc: 0.6665
Epoch 3/10
45568/50000 [==========================>...] - ETA: 19s - loss: 0.4853 - acc: 0.8301

50000/50000 [==============================] - 230s 5ms/step - loss: 0.4848 - acc: 0.8300 - val_loss: 0.6257 - val_acc: 0.8049
Epoch 4/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.4726 - acc: 0.8359 - val_loss: 0.9381 - val_acc: 0.7250
Epoch 5/10
 3328/50000 [>.............................] - ETA: 3:22 - loss: 0.4698 - acc: 0.8386

50000/50000 [==============================] - 231s 5ms/step - loss: 0.4661 - acc: 0.8372 - val_loss: 0.6742 - val_acc: 0.7839
Epoch 6/10
39424/50000 [======================>.......] - ETA: 45s - loss: 0.4558 - acc: 0.8405

50000/50000 [==============================] - 231s 5ms/step - loss: 0.4549 - acc: 0.8402 - val_loss: 0.6539 - val_acc: 0.7992
Epoch 7/10
50000/50000 [==============================] - 230s 5ms/step - loss: 0.4445 - acc: 0.8438 - val_loss: 0.8597 - val_acc: 0.7626
Epoch 8/10
 1792/50000 [>.............................] - ETA: 3:29 - loss: 0.4318 - acc: 0.8488

50000/50000 [==============================] - 231s 5ms/step - loss: 0.4411 - acc: 0.8449 - val_loss: 0.9110 - val_acc: 0.7534
Epoch 9/10
38912/50000 [======================>.......] - ETA: 48s - loss: 0.4270 - acc: 0.8496

50000/50000 [==============================] - 230s 5ms/step - loss: 0.4300 - acc: 0.8489 - val_loss: 0.8725 - val_acc: 0.7639
Epoch 10/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.4231 - acc: 0.8523 - val_loss: 0.8620 - val_acc: 0.7593


In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
model.save_weights("DNST_model_4.h5")
files.download('DNST_model_4.h5')


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 230s 5ms/step - loss: 0.4169 - acc: 0.8525 - val_loss: 0.6206 - val_acc: 0.8179
Epoch 2/10
19200/50000 [==========>...................] - ETA: 2:13 - loss: 0.4038 - acc: 0.8574

50000/50000 [==============================] - 231s 5ms/step - loss: 0.4095 - acc: 0.8564 - val_loss: 0.6074 - val_acc: 0.8140
Epoch 3/10
45568/50000 [==========================>...] - ETA: 19s - loss: 0.4026 - acc: 0.8591

50000/50000 [==============================] - 230s 5ms/step - loss: 0.4025 - acc: 0.8589 - val_loss: 0.6925 - val_acc: 0.8034
Epoch 4/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.3961 - acc: 0.8608 - val_loss: 0.6521 - val_acc: 0.8080
Epoch 5/10
 3328/50000 [>.............................] - ETA: 3:22 - loss: 0.3888 - acc: 0.8606

50000/50000 [==============================] - 231s 5ms/step - loss: 0.3898 - acc: 0.8637 - val_loss: 0.6647 - val_acc: 0.8102
Epoch 6/10
39424/50000 [======================>.......] - ETA: 45s - loss: 0.3795 - acc: 0.8668

50000/50000 [==============================] - 230s 5ms/step - loss: 0.3798 - acc: 0.8667 - val_loss: 0.6948 - val_acc: 0.8092
Epoch 7/10
50000/50000 [==============================] - 230s 5ms/step - loss: 0.3784 - acc: 0.8658 - val_loss: 1.2420 - val_acc: 0.7099
Epoch 8/10
 1792/50000 [>.............................] - ETA: 3:29 - loss: 0.3502 - acc: 0.8750

50000/50000 [==============================] - 231s 5ms/step - loss: 0.3705 - acc: 0.8698 - val_loss: 0.6475 - val_acc: 0.8135
Epoch 9/10
38912/50000 [======================>.......] - ETA: 48s - loss: 0.3587 - acc: 0.8752

50000/50000 [==============================] - 231s 5ms/step - loss: 0.3626 - acc: 0.8742 - val_loss: 0.6815 - val_acc: 0.8031
Epoch 10/10
50000/50000 [==============================] - 230s 5ms/step - loss: 0.3586 - acc: 0.8748 - val_loss: 1.3921 - val_acc: 0.6852


In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
model.save_weights("DNST_model_5.h5")
files.download('DNST_model_5.h5')


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.3574 - acc: 0.8739 - val_loss: 0.7987 - val_acc: 0.7826
Epoch 2/10
19200/50000 [==========>...................] - ETA: 2:13 - loss: 0.3349 - acc: 0.8788

50000/50000 [==============================] - 231s 5ms/step - loss: 0.3482 - acc: 0.8757 - val_loss: 0.5768 - val_acc: 0.8316
Epoch 3/10
45568/50000 [==========================>...] - ETA: 19s - loss: 0.3439 - acc: 0.8791

50000/50000 [==============================] - 231s 5ms/step - loss: 0.3442 - acc: 0.8791 - val_loss: 0.7516 - val_acc: 0.7954
Epoch 4/10
50000/50000 [==============================] - 231s 5ms/step - loss: 0.3404 - acc: 0.8806 - val_loss: 0.7898 - val_acc: 0.7950
Epoch 5/10
 3328/50000 [>.............................] - ETA: 3:23 - loss: 0.3245 - acc: 0.8846

50000/50000 [==============================] - 230s 5ms/step - loss: 0.3384 - acc: 0.8798 - val_loss: 0.6325 - val_acc: 0.8126
Epoch 6/10
39424/50000 [======================>.......] - ETA: 45s - loss: 0.3303 - acc: 0.8823

50000/50000 [==============================] - 230s 5ms/step - loss: 0.3327 - acc: 0.8817 - val_loss: 0.6136 - val_acc: 0.8248
Epoch 7/10
50000/50000 [==============================] - 228s 5ms/step - loss: 0.3258 - acc: 0.8849 - val_loss: 0.5812 - val_acc: 0.8278
Epoch 8/10
 1792/50000 [>.............................] - ETA: 3:25 - loss: 0.3332 - acc: 0.8711

50000/50000 [==============================] - 228s 5ms/step - loss: 0.3228 - acc: 0.8869 - val_loss: 0.7308 - val_acc: 0.8034
Epoch 9/10
38912/50000 [======================>.......] - ETA: 47s - loss: 0.3171 - acc: 0.8875

50000/50000 [==============================] - 228s 5ms/step - loss: 0.3217 - acc: 0.8858 - val_loss: 0.7032 - val_acc: 0.7960
Epoch 10/10
50000/50000 [==============================] - 228s 5ms/step - loss: 0.3176 - acc: 0.8868 - val_loss: 0.5845 - val_acc: 0.8346


In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 20s 2ms/step
Test loss: 0.5845345138788224
Test accuracy: 0.8346


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model_final.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
files.download('DNST_model_final.h5')